In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fco2dataset.ucruise import divide_cruise, df_to_numpy

NUM_BINS = 64
WINDOW_LEN = 5

COLS = ['sal', 'sst_deg_c', 'pppp_hpa', 'woa_sss', 'ncep_slp_hpa',
       'dist_to_land_km', 'fco2rec_uatm', 'fco2rec_flag', 'time', 'year',
       'chl_globcolour', 'chl_globcolour_uncert', 'chl_globcolour_flags',
       'time_chlgc', 'lat_chlgc', 'lon_chlgc', 'temp_soda', 'salt_soda',
       'mld_dens_soda', 'lon_soda', 'lat_soda', 'depth_soda', 'time_soda',
       'ssh_adt', 'ssh_sla', 'lat_sshcm', 'lon_sshcm', 'time_sshcm', 'sst_cci',
       'sst_cci_uncertainty', 'ice_cci', 'time_sstcci', 'lat_sstcci',
       'lon_sstcci', 'sss_cci', 'sss_cci_random_error', 'lat_ssscci',
       'lon_ssscci', 'time_ssscci', 'interpolated']

predictors = ['sst_cci', 'sss_cci', 'pppp_hpa', 'woa_sss', 'ncep_slp_hpa', 'dist_to_land_km', 
              'chl_globcolour', 'temp_soda', 'salt_soda', 'mld_dens_soda', 
              'ssh_adt', 'ssh_sla', 'ice_cci']

dss = []
total_cruises = 0
total_segments = 0
for year in range(1982, 2022):
    # Load interpolated data
    fname_interp_socat = f'../data/SOCATv2024_interpolated2/.gridded_2d_ocean_data_for_ML/SOCATv2024-cruise_tracks_interp-collocated/socat2024-interp_tracks-collocated-{year}.pq'
    df_interp_socat = pd.read_parquet(fname_interp_socat)
    df_interp_socat.set_index(['expocode', 'time_1d', 'lat', 'lon'], inplace=True)
    df_interp_socat = df_interp_socat.groupby(level=[0,1,2,3]).mean()

    cruises = len(df_interp_socat.index.unique(level=0))
    total_cruises += cruises
    print(f"number of cruises in {year}: {cruises}")

    # read raw socat data
    fname_socat = f'../data/SOCATv2024_raw-collocated-1982_2021/SOCATv2024_raw_r20250307-1982_2021/SOCATv2024v_collocated-{year}.pq'
    df_socat = pd.read_parquet(fname_socat)
    df_socat['time_1d'] = df_socat['time'].dt.round('D')
    df_socat.set_index(['expocode', 'time_1d','lat', 'lon'], inplace=True)
    df_socat['interpolated'] = False

    df = pd.concat([df_socat, df_interp_socat], axis=0)
    df = df.reindex(columns=COLS, fill_value=np.nan)
    # print(df.shape)
    df2 = df.groupby(level=0).apply(
        lambda cruise: divide_cruise(
            cruise,
            num_windows=NUM_BINS,
            len_window=WINDOW_LEN,
            max_time_delta=pd.Timedelta(days=2)
        )
    )

    segments = df2.groupby([pd.Grouper(level=0), 'segment_id']).size().size
    total_segments += segments
    print(f"number of segments in {year}: {segments}")
    
    ds, expomap = df_to_numpy(df2, NUM_BINS, predictors)
    dss.append(ds)
    print(f"finished year {year}")

ds = np.concatenate(dss, axis=1)
print(f"total number of cruises: {total_cruises}")
print(f"total number of segments: {total_segments}")
print(f"total number of samples: {ds.shape[1]}")
ds.shape

number of cruises in 1982: 8
number of segments in 1982: 112
finished year 1982
number of cruises in 1983: 7
number of segments in 1983: 150
finished year 1983
number of cruises in 1984: 5
number of segments in 1984: 137
finished year 1984
number of cruises in 1985: 9
number of segments in 1985: 209
finished year 1985
number of cruises in 1986: 11
number of segments in 1986: 324
finished year 1986
number of cruises in 1987: 19
number of segments in 1987: 506
finished year 1987
number of cruises in 1988: 13
number of segments in 1988: 332
finished year 1988
number of cruises in 1989: 15
number of segments in 1989: 302
finished year 1989
number of cruises in 1990: 15
number of segments in 1990: 321
finished year 1990
number of cruises in 1991: 28
number of segments in 1991: 871
finished year 1991
number of cruises in 1992: 35
number of segments in 1992: 944
finished year 1992
number of cruises in 1993: 33
number of segments in 1993: 1117
finished year 1993
number of cruises in 1994: 43
n

(14, 117337, 65)

In [ ]:
# save as numpy array
np.save('../data/training_data/trainds_100km.npy', ds)
# save as dataframe
columns = ['fco2rec_uatm', 'sst_cci', 'sss_cci', 'pppp_hpa', 'woa_sss', 'ncep_slp_hpa', 'dist_to_land_km', 
           'chl_globcolour', 'temp_soda', 'salt_soda', 'mld_dens_soda', 
           'ssh_adt', 'ssh_sla', 'ice_cci']
segment_ix = np.repeat(np.arange(0, ds.shape[1]), ds.shape[2])
bin_ix = np.tile(np.arange(0, ds.shape[2]), ds.shape[1])
index = pd.MultiIndex.from_arrays([segment_ix, bin_ix], names=['segment', 'bin'])
df = pd.DataFrame(ds.reshape(-1, len(columns)), columns=columns, index=index)
df.to_parquet('../data/training_data/traindf_100km.pq')

In [ ]:

import numpy as np
# read the numpy array
ds = np.load('../data/training_data/trainds_100km.npy')
print(ds.shape)

col_map = dict(zip(columns, range(len(columns))))

# ESA-CCI > SODA 
# merge salt_soda and sss_cci, where sss_cci has priority, use salt_soda where sss_cci is nan
salt_soda = ds[col_map['salt_soda']]
sss_cci = ds[col_map['sss_cci']]
mask = np.isnan(sss_cci)
ds[col_map['sss_cci'], mask] = salt_soda[mask]

(14, 117337, 65)


In [ ]:
from dataset_create.ucruise import filter_nans
X, y = filter_nans(ds, ds[col_map['fco2rec_uatm']], predictors, col_map)

In [ ]:
fig, axs = plt.subplots(X.shape[0] + 1, 1, figsize=(5*(X.shape[0] + 1), 10), sharex=True)

plt.xlim((0, NUM_BINS))
titles = ['fco2', 'temperature', 'salinity']
seg = 20
axs[0].plot(y[seg])
print(y[seg])
axs[0].set_title(titles[0])
for i in range(X.shape[0]):
    axs[i + 1].plot(X[i, seg])
    axs[i + 1].set_title(titles[i + 1])
plt.show()

NameError: name 'plt' is not defined